<img src="logo.png">

# Separación y entrenamiento

In [ ]:
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn import datasets

In [ ]:
boston = datasets.load_boston()
boston.keys()


In [ ]:
print(boston["DESCR"])

In [ ]:
boston["data"]

In [ ]:
def rmse(objetivo, estimaciones):
    return np.sqrt(metrics.mean_squared_error(objetivo, estimaciones)
                  )

def adjusted_r2(objetivo, estimaciones, n, k):
    r2 = metrics.r2_score(objetivo, estimaciones)
    return 1 - (1-r2)*(n-1) / (n - k - 1)

def evaluar_modelo(objetivo, estimaciones, n, k):
    return {
        "rmse": rmse(objetivo, estimaciones),
        "mae": metrics.mean_absolute_error(objetivo, estimaciones),
        "adjusted_r2": adjusted_r2(objetivo, estimaciones, n, k)
           }

In [ ]:
modelo_ols = LinearRegression()

modelo_ols.fit(X=boston["data"], y=boston["target"])

modelo_ols_preds = modelo_ols.predict(boston["data"])

In [ ]:
RESULTADOS = {}

In [ ]:
N = boston["data"].shape[0]

RESULTADOS["ols"] = evaluar_modelo(
    boston["target"],
    modelo_ols_preds,
    N,
    len(modelo_ols.coef_)
)

RESULTADOS

Hasta este momento hemos usado todos los datos como entrenamiento, de modo que posiblemente tengamos problemas de sobreajuste y por lo tanto tengamos errores si introducimos datos nuevos. Lo que haremos entonces es dividir la tabla para reentrenar el modelo.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_test_split?

In [ ]:
boston["data"].shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
     boston["data"], boston["target"],
     test_size=0.33, random_state=13
)

In [ ]:
print(X_train.shape, y_train.shape)

In [ ]:
print(X_test.shape, y_test.shape)

In [ ]:
# Preparamos el modelo
modelo_ols = LinearRegression()

In [ ]:
# Ajustamos el modelo entrenándolo
modelo_ols.fit(X=X_train, y=y_train)
modelo_ols_train_preds = modelo_ols.predict(X_train)

# Obtenemos los resultados del modelo recién entrenado
RESULTADOS["ols_train"] = evaluar_modelo(
    y_train,
    modelo_ols_train_preds,
    X_train.shape[0],
    len(modelo_ols.coef_)
)

Hasta este momento, los coeficientes de la regresión en el modelo entrenado están guardados en ``modelo_ols.intercept_`` y ``modelo_ols.coef_``:

In [ ]:
{"intercepcion":modelo_ols.intercept_,"coeficientes":modelo_ols.coef_}

Utilizando estos parámetros, aplicamos a ``X_test``, el conjunto de prueba, para obtener las predicciones.

In [ ]:
modelo_ols_test_preds = modelo_ols.predict(X_test)

RESULTADOS["ols_test"] = evaluar_modelo(
    y_test,
    modelo_ols_test_preds,
    X_test.shape[0],
    len(modelo_ols.coef_)
)

In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame(RESULTADOS)

Vemos que al separar los datos de entrenamiento y los de test se obtiene un resultado peor al evaluar los datos de train.

Podríamos parar aquí y decir *"El error RMSE de mi modelo es 4.787026"*, y podríamos pensar que esta todo bien ya que no hemos entrenado el modelo en los datos que hemos usado para evaluarlo.

Pero estaríamos en un grave error. ¿Por qué? 

Recordemos que hemos usado un `random_state=13` para la función `train_test_split` que garantiza que la separación de entrenamiento y test sea siempre la misma. Podemos usar cualquier número para este argumento.

Qué pasa si usamos por ejemplo `random_state=42`?

In [ ]:
RESULTADOS = {}

X_train, X_test, y_train, y_test = train_test_split(
     boston["data"], boston["target"],
     test_size=0.33, random_state=42
)


modelo_ols = LinearRegression()
modelo_ols.fit(X=X_train, y=y_train)
modelo_ols_train_preds = modelo_ols.predict(X_train)
modelo_ols_test_preds = modelo_ols.predict(X_test)


RESULTADOS["ols_train"] = evaluar_modelo(
    y_train,
    modelo_ols_train_preds,
    X_train.shape[0],
    len(modelo_ols.coef_)
)

RESULTADOS["ols_test"] = evaluar_modelo(
    y_test,
    modelo_ols_test_preds,
    X_test.shape[0],
    len(modelo_ols.coef_)
)


pd.DataFrame(RESULTADOS)

¡El error en los datos de test es menor que en los de entrenamiento! ¿Por qué? Sencillamente, por que ha dado la casualidad de que hemos separado los datos de una forma que los datos de test son muy fáciles de estimar.

Para ver la magnitud del error en el que estamos cayendo al hacer una sola separación entre test y entrenamiento, vamos a probar un monton de semillas y ver cual es el rango del error que se puede obtener 

In [ ]:
model=LinearRegression()
results = []
def test_seed(seed):
    X_train, X_test, y_train, y_test = train_test_split(
     boston["data"], boston["target"],
     test_size=0.33, random_state=seed
    )
    test_preds = model.fit(X_train, y_train).predict(X_test)
    seed_rmse = rmse(y_test, test_preds)
    results.append([seed_rmse, seed])

In [ ]:
for i in range(1000):
    test_seed(i)

In [ ]:
results[:5]

In [ ]:
results_sorted = sorted(results, key=lambda x: x[0], reverse=False)

In [ ]:
results_sorted[0]

In [ ]:
results_sorted[-1]

**Validación Cruzada (Cross Validation)**

Vemos que entre la semilla con menor error de test y la semilla con mayor error hay una diferencia casi del doble!

Una forma de evitar el cometer este error es mediante la **Validación cruzada**

![cross_val](https://cdn-images-1.medium.com/max/1600/1*J2B_bcbd1-s1kpWOu_FZrg.png)

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
cross_val_score?

In [ ]:
modelo_ols = LinearRegression()
X = boston["data"]
y = boston["target"]

resultados_validación_cruzada = cross_val_score(
    estimator=modelo_ols, 
    X=X,
    y=y,
    scoring="neg_mean_squared_error", 
    cv=10
)

In [ ]:
resultados_validación_cruzada

In [ ]:
resultados_validación_cruzada.mean()

In [ ]:
def rmse_cross_val(estimator, X, y):
    y_pred = estimator.predict(X)
    return np.sqrt(metrics.mean_squared_error(y, y_pred))

In [ ]:
resultados_cv = []
for i in range(10,200):
    cv_rmse = cross_val_score(
        estimator=modelo_ols, 
        X=X,
        y=y,
        scoring=rmse_cross_val, 
        cv=i
    ).mean()
    resultados_cv.append(cv_rmse)

In [ ]:
#%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
plt.plot(resultados_cv)

In [ ]:
cross_validate?

In [ ]:
from sklearn.model_selection import cross_validate
scoring = {"mae": "neg_mean_absolute_error", "rmse": rmse_cross_val}
estimator = modelo_ols
scores = cross_validate(estimator, boston["data"],
                        boston["target"], scoring=scoring,
                         cv=100, return_train_score=True)

In [ ]:
pd.DataFrame(scores).mean()